<a href="https://colab.research.google.com/github/CNIU1997/BA_888/blob/master/BA888_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## upload files and import library
from google.colab import files
uploaded = files.upload()

import pandas as pd
import numpy as np

Saving train_data.csv to train_data.csv


In [0]:
train = pd.read_csv('train_data.csv',index_col=0)
test = pd.read_csv('test_data.csv',index_col=0)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,224,225,226) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, comp

In [0]:
## base on Kiki's feature selection results and data prepare process
from sklearn.preprocessing import StandardScaler
features = ['Effect_of_forex_changes_on_cash','EPS_Diluted','Revenue_Growth','Gross_Profit_Growth','Net_cash_flow_/_Change_in_cash','EPS','Financing_Cash_Flow',
'Weighted_Average_Shares_Growth','EPS_Diluted_Growth','Book_Value_per_Share_Growth','Operating_Cash_Flow_growth','Receivables_growth',
'Weighted_Average_Shs_Out','EPS_Growth','Weighted_Average_Shares_Diluted_Growth','SG&A_Expenses_Growth','Operating_Cash_Flow','Retained_earnings_(deficit)',
'Operating_Income_Growth','Operating_Cash_Flow_per_Share']
# Separating out the features
x_train = train.loc[:, features].values
x_test= test.loc[:,features].values
# transfer into dataframe
x_train = pd.DataFrame(data=x_train,columns=[features])
x_test = pd.DataFrame(data=x_test,columns=[features])
# transfer to numeric for X-train and X-test
cols = x_train.select_dtypes(exclude=['float']).columns
cols2 = x_test.select_dtypes(exclude=['float']).columns
x_train[cols] = x_train[cols].apply(pd.to_numeric, downcast='float', errors='coerce')
x_test[cols2] = x_test[cols2].apply(pd.to_numeric, downcast='float', errors='coerce')
# Separating out the target
y_train = train.loc[:,['Class']].values
y_test = test.loc[:,['Class']].values
# transfer into dataframe
y_train= pd.DataFrame(data=y_train,columns=["Class"])
y_test= pd.DataFrame(data=y_test,columns=["Class"])
# transfer to numeric for y-train
y_train['Class'] =y_train['Class'].apply(pd.to_numeric, downcast='float', errors='coerce')
# replace x NAs with 0
x_train.fillna(0, inplace=True)
x_test.fillna(0, inplace=True)
# drop NAs in y
y_train[pd.isnull(y_train).any(axis=1)]
y_train.dropna(inplace = True)
# drop corresponding x rows
x_train.drop(x_train.index[[7928,12726,12727,17688,17689]],inplace=True)
# winsorize the x-train 
from scipy.stats.mstats import winsorize
# remove outliers 5% higher than the max and 5% lower than the min using for loop and winsorize
for col in x_train: 
     x_train[col] = winsorize(x_train[col], limits=[0.05,0.05])
# Standardizing the features
from sklearn.preprocessing import StandardScaler
x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components=6)
principalComponents = pca.fit_transform(x_train)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2','principal component 3','principal component 4','principal component 5','principal component 6'])

In [0]:
finalDf = pd.concat([principalDf, y_train.reindex(principalDf.index)], axis=1, sort=False)

In [0]:
pca.explained_variance_ratio_

array([0.22191049, 0.17434041, 0.10907665, 0.07634877, 0.0618388 ,
       0.05449552], dtype=float32)

In [0]:
# sns.pairplot(data=finalDf, hue="Class", height=8,corner=True)
# sns.set_context("notebook", font_scale=2)

In [0]:
# import KMeans
from sklearn.cluster import KMeans
# Convert DataFrame to matrix
mat = principalDf.values
# Using sklearn, change clusters accordingly
km = KMeans(n_clusters=2)
km.fit(mat)
# Get cluster assignment labels
labels = km.labels_
labels

array([1, 0, 1, ..., 0, 0, 0], dtype=int32)

In [0]:
finalDf['Group']=labels
finalDf

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,Class,Group
0,5.547273,-1.535990,3.023848,1.107467,-0.434413,1.906527,0.0,1
1,0.805211,4.728044,1.157721,-1.917841,1.191317,2.340343,0.0,0
2,3.540299,-0.746336,2.824049,0.120840,-0.396440,1.975510,1.0,1
3,2.501961,0.560707,-0.895622,-0.659572,-1.558384,-0.240660,1.0,1
4,5.314578,-1.610777,3.701818,1.708428,0.731729,-0.381869,1.0,1
...,...,...,...,...,...,...,...,...
17680,-0.711430,-1.120945,-0.826374,-0.233552,0.122353,-0.240608,1.0,0
17681,-0.515187,-1.927233,-1.221475,-0.204014,-0.066959,-0.303945,0.0,0
17682,-0.563739,-0.677748,-1.178605,0.041297,0.090509,-0.068151,1.0,0
17683,-0.315605,-1.164640,-1.188137,-0.209941,-0.871819,-0.925204,0.0,0


In [0]:
finalDf['Match'] = np.where(finalDf['Group'] == finalDf['Class'], 1,0)
# The variable total adds up the total number of matches.
Total = finalDf['Match'].sum()
print (Total/17685)

0.5014984450098954
